## Evapotranspiration and balance calculations at the atmospheric boundary

This is an extension of our excercise on evapotranspiration (ET) calculations.

(cc) conrad.jackisch@tbt.tu-freiberg.de, May 04, 2022

In [1]:
%pylab inline
import pandas as pd
import seaborn as sns
import plotly.express as px

import plotly.io as pio
pio.renderers.default='iframe' #for windows use 'notebook' instead
from plotly.subplots import make_subplots
import plotly.graph_objects as go

sns.set_style('whitegrid', {'grid.linestyle': u'--'})
matplotlib.rc('pdf', fonttype=42)

Populating the interactive namespace from numpy and matplotlib


In [2]:
import dwd
import pyeto as pt

## DWD Stationen mit unterschiedlicher Datenverfügbarkeit

In [3]:
# DWD Stations
dwd_stats = dwd.dwd_stations()
dwd_stats['StatID'] = dwd_stats.index
dwd_stats['Active'] = np.log(((pd.Timestamp.now()-dwd_stats['bis_datum']).values).astype(float)/86400000000000.)

In [4]:
px.set_mapbox_access_token('pk.eyJ1IjoiY29qYWNrIiwiYSI6IkRTNjV1T2MifQ.EWzL4Qk-VvQoaeJBfE6VSA')

fig = px.scatter_mapbox(dwd_stats, lat='geoBreite', lon='geoLaenge',  color='Active', hover_name='Stationsname', hover_data=['StatID','von_datum', 'bis_datum'],
                  color_continuous_scale=px.colors.sequential.YlOrRd_r, zoom=5)
fig.show()

You can download the DWD data with my toolbox by simply defining the station through its name. You can select to download only recent data `histo = False` or the complete set provided on the DWD server `histo = False`. You can choose daily `rss='1D'` and hourly data `rss='1H'`.

In [5]:
# load station weather data
# fill missing values at Dresden Klotzsche with data from Dresden Strehlen
weather = dwd.resample_DWD('Klotzsche',histo=True,rss='1D')
weather2 = dwd.resample_DWD('Strehlen',histo=True,rss='1D')

firstitem = True
for k in weather.columns:
    if firstitem:
        weatherx = pd.concat([weather[k],weather2[k]],axis=1).fillna(method='bfill',axis=1).iloc[:,0]
        firstitem = False
    else:
        weatherx = pd.concat([weatherx,pd.concat([weather[k],weather2[k]],axis=1).fillna(method='bfill',axis=1).iloc[:,0]],axis=1)
weather = weatherx

In [6]:
weather.head()

,T,Tmin,Tmax,Prec,Rad,Rs,RH,u2,u2mx,vap,aP
1934-01-01,0.5,0.2,0.7,0.2,NaN,NaN,97.0,NaN,NaN,6.4,1008.6
1934-01-02,-0.1,-0.8,0.9,0.1,NaN,NaN,90.0,NaN,NaN,5.9,1006.2
1934-01-03,-0.7,-1.0,-0.1,0.0,NaN,NaN,85.0,NaN,NaN,5.2,1001.6
1934-01-04,-1.6,-3.5,-0.3,0.0,NaN,NaN,82.0,NaN,NaN,4.8,1001.4
1934-01-05,0.9,-3.8,2.8,10.1,NaN,NaN,92.0,NaN,NaN,6.3,996.0


In [7]:
# Saturated vapour pressure based on Tmean
weather['vs_Tmean'] = 0.61078 * np.exp(17.1 * weather['T'] / (weather['T'] + 235.)) 

# Saturated vapour pressure based on Tmin/Tmax
vs_Tmax = 0.61078 * np.exp(17.1 * weather['Tmax'] / (weather['Tmax'] + 235.))  
vs_Tmin = 0.61078 * np.exp(17.1 * weather['Tmin'] / (weather['Tmin'] + 235.)) 
weather['vas'] = (vs_Tmax + vs_Tmin) / 2.

weather['EToPM'] = pt.fao56_penman_monteith(weather.Rad.values*0.01,weather['T'].values+273.15,weather.u2.values,pt.svp_from_t(weather['T'].values),weather.vas*0.1,pt.delta_svp(weather['T'].values),pt.psy_const(0.1*weather.aP))
weather['EToHG'] = pt.hargreaves(weather.Tmin.values,weather.Tmax.values,weather['T'].values,pt.et_rad(52. * np.pi/180.,pt.sol_dec(weather.index.dayofyear.values),pt.sunset_hour_angle(52. * np.pi/180.,pt.sol_dec(weather.index.dayofyear.values)),pt.inv_rel_dist_earth_sun(weather.index.dayofyear.values)))

weather['EToSJ'], weather['EToPM2'], weather['EToPT'] = dwd.ET_SzilagyiJozsa(weather)


https://www.umwelt.sachsen.de/umwelt/infosysteme/hwims/portal/web/wasserstand-pegel-566010

| Pegel | Berthelsdorf |
| --- | --- |
| Zuständig | LfULG - LHWZ |
| Gewässer | Freiberger Mulde |
| Flussgebiet | Mulde |
| Einzugsgebiet | 244,0 km2 |
| Pegelnullpunkt b | 376,61 m ü. Bezugshorizont |


| Hydrologische | Hauptwerte | a |
| --- | --- | --- |
| Mittlerer Niedrig-W/Q | 42 cm | 0,655 m3/s |
| Mittlerer W/Q | 60 cm | 3,48 m3/s |
| Mittlerer Hoch-W/Q | 136 cm | 35,2 m3/s |
| Höchster Hoch-W/Q | 385 cm | 360 m3/s |

In [8]:
# load daily drainage at gauge Berthelsdorf
bertq = pd.read_csv('BerthelsdorfQ.csv',index_col=0)
bertq.index = pd.to_datetime(bertq.index)

In [25]:
bertWB = pd.concat([weather.Prec,(bertq.Durchfluss*(86400./2440000.)),weather.EToHG],axis=1)
fig = px.line(bertWB.resample('1Y').sum())
fig.update_layout({'template': 'none','title': 'Berthelsdorf', 'yaxis_title': '(mm/year)'})

In [11]:
fig = px.line((bertxq.Durchfluss*86400./244000.).resample('1Y').sum())
fig.update_layout({'template': 'none','title': 'Berthelsdorf', 'yaxis_title': 'Q (mm/year)'})

count    1093.000000
mean      314.963850
std       434.335581
min         3.895082
25%       101.626230
50%       200.065574
75%       323.291803
max      8604.590164
Name: Durchfluss, dtype: float64

In [10]:
# plot a variable
fig = px.line(weather[['EToHG', 'EToPM', 'EToSJ', 'EToPM2', 'EToPT']])
fig.update_layout({'template': 'none','title': 'Dresden Klotzsche', 'yaxis_title': 'Prec (mm/day)'})